---
title: IDs from Wind
output-file: wind.html
---

## Setup

Need to run command in shell first as `pipeline` is project-specific command

```{sh}
kedro pipeline create wind
```

In [4]:
#| default_exp pipelines/wind/pipeline


In [5]:
#| hide
%load_ext autoreload
%autoreload 2

In [6]:
#| export
import polars as pl

In [14]:
# | eval: false
from ids_finder.utils.basic import load_catalog

In [15]:
catalog = load_catalog('../..')

jno_start_date = catalog.load("params:jno_start_date")
jno_end_date = catalog.load("params:jno_end_date")
trange = [jno_start_date, jno_end_date]

[10/28/23 09:23:14] WARNING  /Users/zijin/miniforge3/envs/cool_planet/lib/python3.10/site-packages/ke ]8;id=449562;file:///Users/zijin/miniforge3/envs/cool_planet/lib/python3.10/site-packages/astropy/logger.py\logger.py]8;;\:]8;id=580155;file:///Users/zijin/miniforge3/envs/cool_planet/lib/python3.10/site-packages/astropy/logger.py#205\205]8;;\
                             dro/io/partitioned_dataset.py:200: KedroDeprecationWarning:                           
                             'PartitionedDataset' has been moved to `kedro-datasets` and will be                   
                             removed in Kedro 0.19.0.                                                              
                               warnings.warn(                                                                      
                                                                                                                   

                    WARNING  /Users/zijin/miniforge3/envs/cool_planet/lib/python3.10/site-packages/ke ]8;id=883252;file:///Users/zijin/miniforge3/envs/cool_planet/lib/python3.10/site-packages/astropy/logger.py\logger.py]8;;\:]8;id=939990;file:///Users/zijin/miniforge3/envs/cool_planet/lib/python3.10/site-packages/astropy/logger.py#205\205]8;;\
                             dro_datasets/polars/lazy_polars_dataset.py:14: KedroDeprecationWarning:               
                             'AbstractVersionedDataSet' has been renamed to                                        
                             'AbstractVersionedDataset', and the alias will be removed in Kedro                    
                             0.19.0                                                                                
                               from kedro.io.core import (                                                         
                                                                                                                   

                    INFO     Loading data from 'params:jno_start_date' (MemoryDataset)...       ]8;id=956851;file:///Users/zijin/miniforge3/envs/cool_planet/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=144433;file:///Users/zijin/miniforge3/envs/cool_planet/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'params:jno_end_date' (MemoryDataset)...         ]8;id=480640;file:///Users/zijin/miniforge3/envs/cool_planet/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=684216;file:///Users/zijin/miniforge3/envs/cool_planet/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

## Magnetic field data pipeline

- For convenience, we choose magnetic field data in **GSE** coordinate system
- The `fgs` data are in 3-4s resolution

In [9]:
# | export
import pyspedas
from ids_finder.utils.basic import cdf2pl, pmap, resample

In [18]:
#| export
def download_mag_data(trange: list[str], datatype) -> list[str]:
    files = pyspedas.wind.mfi(trange, datatype=datatype, downloadonly=True)
    return files

def load_mag_data(
    start: str = None,
    end: str = None,
    trange: list[str] = None,
    datatype="h4-rtn",
):
    if trange is None:
        trange = [start, end]

    files = download_mag_data(trange=trange, datatype=datatype)
    var_names = ["BRTN", "BF1"]
    df: pl.LazyFrame = pl.concat(files | pmap(cdf2pl, var_names=var_names))

    return df

In [12]:
def preprocess_mag_data(
    raw_data: pl.LazyFrame,
) -> pl.LazyFrame:
    """
    Preprocess the raw dataset (only minor transformations)

    - Downsample the data to a given time resolution
    - Applying naming conventions for columns
    """
    name_mapping = {
        "BRTN_0": "B_x",
        "BRTN_1": "B_y",
        "BRTN_2": "B_z",
        "BF1": "B_mag",
    }

    return raw_data.rename(name_mapping)

In [ ]:
# | export
from datetime import timedelta
from ids_finder.utils.basic import partition_data_by_year

In [ ]:
#| export
def process_mag_data(
    raw_data: pl.LazyFrame,
    ts: str = None,  # time resolution
    coord: str = None,
):
    """
    Corresponding to primary data layer, where source data models are transformed into domain data models

    - Partitioning data, for the sake of memory
    """
    
    every = timedelta(seconds=ts)
    period = 2 * every
    
    return partition_data_by_year(raw_data)

In [ ]:
#| export
def create_pipeline(
    sat_id="sta",
    tau="60s",
    ts_mag="1s",  # time resolution of magnetic field data
    ts_state="1h",  # time resolution of state data
    **kwargs
):
    pass